In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder 
import boto3
import plotly.express as px
from sklearn.model_selection import train_test_split
import requests
from io import StringIO
from datetime import datetime, timedelta
import time
import sys
import logging

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', handlers=[logging.StreamHandler(sys.stdout)])
logger = logging.getLogger(__name__)

In [ ]:
bucket = "REDACTED_BUCKET"
dax_key = "dax.csv"
def read_data_from_s3(bucket, key):
    s3 = boto3.client("s3")
    obj = s3.get_object(Bucket=bucket, Key=key)
    return pd.read_csv(obj['Body'], parse_dates=['Date'])

df = read_data_from_s3(bucket, dax_key)

In [ ]:
print(df.isnull().sum())
df = df.dropna()

In [ ]:
df.tail()

In [ ]:
len(df.values)

In [ ]:
def one_hot_encode_column(data: pd.DataFrame, col, max_classes=None):
    df = data.copy()
    
    if max_classes is not None:
        dummies = pd.get_dummies(df[col], prefix=col, dtype=int)
         
        missing_cols = set(range(max_classes + 1)) - set(df[col].unique())
        for col_num in missing_cols:
            dummies[col +"_"+ str(col_num)] = 0
        
        # Custom sort function to ensure correct column order
        dummies = dummies[sorted(dummies.columns, key=lambda x: int(x.split("_")[1]))]
    else:
        dummies = pd.get_dummies(df[col], prefix=col, dtype=int)

    df = pd.concat([df, dummies], axis=1)
    if max_classes is None:
        df = df.drop(columns=[col])

    return df

In [ ]:
df["Open_200_MA"] = df["Open"].rolling(window=200).mean()
df["High_200_MA"] = df["High"].rolling(window=200).mean()
df["Low_200_MA"] = df["Low"].rolling(window=200).mean()
df["Close_200_MA"] = df["Close"].rolling(window=200).mean()
df["Open_100_MA"] = df["Open"].rolling(window=100).mean()
df["High_100_MA"] = df["High"].rolling(window=100).mean()
df["Low_100_MA"] = df["Low"].rolling(window=100).mean()
df["Close_100_MA"] = df["Close"].rolling(window=100).mean()
df["Open_10_MA"] = df["Open"].rolling(window=10).mean()
df["High_10_MA"] = df["High"].rolling(window=10).mean()
df["Low_10_MA"] = df["Low"].rolling(window=10).mean()
df["Close_10_MA"] = df["Close"].rolling(window=10).mean()
df = df.dropna()

In [ ]:
df.tail()

In [ ]:
fig = px.line(df, x="Date", y=["Open","Open_200_MA","Open_100_MA","Open_10_MA"])
fig.show()

In [ ]:
import ta


# Calculate RSI
df['RSI_open'] = ta.momentum.rsi(df['Open'])
df['RSI_open'] = (df['RSI_open'] - 50) / 50
df['RSI_diff_open'] = df['RSI_open'].diff()

In [ ]:
fig = px.line(df, x="Date", y=['RSI_open'])
fig.show()

In [ ]:
df['Close_lag1'] = df['Close'].shift(1)
df['Open_lag1'] = df['Open'].shift(1)
df['Low_lag1'] = df['Low'].shift(1)
df['High_lag1'] = df['High'].shift(1)

df['Close_lag1_200_MA'] = df['Close_200_MA'].shift(1)
df['Open_lag1_200_MA'] = df['Open_200_MA'].shift(1)
df['Low_lag1_200_MA'] = df['Low_200_MA'].shift(1)
df['High_lag1_200_MA'] = df['High_200_MA'].shift(1)

df['Close_lag1_100_MA'] = df['Close_100_MA'].shift(1)
df['Open_lag1_100_MA'] = df['Open_100_MA'].shift(1)
df['Low_lag1_100_MA'] = df['Low_100_MA'].shift(1)
df['High_lag1_100_MA'] = df['High_100_MA'].shift(1)

df['Close_lag1_10_MA'] = df['Close_10_MA'].shift(1)
df['Open_lag1_10_MA'] = df['Open_10_MA'].shift(1)
df['Low_lag1_10_MA'] = df['Low_10_MA'].shift(1)
df['High_lag1_10_MA'] = df['High_10_MA'].shift(1)


In [ ]:
df["Open_Close"] = (df["Open"]- df["Close_lag1"]) / df["Open"]
df["Open_Open"] = (df["Open"]- df["Open_lag1"]) / df["Open"]
df["Open_Low"] = (df["Open"]- df["Low_lag1"]) / df["Open"]
df["Open_High"] = (df["Open"]- df["High_lag1"]) / df["Open"]

df["Open_Close_200_MA"] = (df["Open"]- df["Close_lag1_200_MA"]) / df["Open"]
df["Open_Open_200_MA"] = (df["Open"]- df["Open_lag1_200_MA"]) / df["Open"]
df["Open_Low_200_MA"] = (df["Open"]- df["Low_lag1_200_MA"]) / df["Open"]
df["Open_High_200_MA"] = (df["Open"]- df["High_lag1_200_MA"]) / df["Open"]

df["Open_Close_100_MA"] = (df["Open"]- df["Close_lag1_100_MA"]) / df["Open"]
df["Open_Open_100_MA"] = (df["Open"]- df["Open_lag1_100_MA"]) / df["Open"]
df["Open_Low_100_MA"] = (df["Open"]- df["Low_lag1_100_MA"]) / df["Open"]
df["Open_High_100_MA"] = (df["Open"]- df["High_lag1_100_MA"]) / df["Open"]

df["Open_Close_10_MA"] = (df["Open"]- df["Close_lag1_10_MA"]) / df["Open"]
df["Open_Open_10_MA"] = (df["Open"]- df["Open_lag1_10_MA"]) / df["Open"]
df["Open_Low_10_MA"] = (df["Open"]- df["Low_lag1_10_MA"]) / df["Open"]
df["Open_High_10_MA"] = (df["Open"]- df["High_lag1_10_MA"]) / df["Open"]

In [ ]:
df['EMA_Close'] = ta.trend.ema_indicator(df['Close'])
df['EMA_Ratio_Close'] = (df['Close'] - df['EMA_Close']) / df['Close']
bollinger = ta.volatility.BollingerBands(close=df['Close'])
df['BB_Bandwidth'] = (bollinger.bollinger_hband() - bollinger.bollinger_lband()) / bollinger.bollinger_mavg()
df['BB_Percent'] = (df['Close'] - bollinger.bollinger_lband()) / (bollinger.bollinger_hband() - bollinger.bollinger_lband())/2
stochastic = ta.momentum.stoch(df['High'], df['Low'], df['Close'])
df['Stochastic_Scaled'] = stochastic / 100.0
df['EMA_Ratio_Close'] = df['EMA_Ratio_Close'].shift(1)
df['BB_Bandwidth'] = df['BB_Bandwidth'].shift(1)
df['BB_Percent'] = df['BB_Percent'].shift(1)
df['Stochastic_Scaled']  = df['Stochastic_Scaled'].shift(1)




In [ ]:
df['SMA_5'] = ta.trend.SMAIndicator(close=df['Close'], window=5).sma_indicator()
df['SMA_10'] = ta.trend.SMAIndicator(close=df['Close'], window=10).sma_indicator()
df['MA_Crossover_Signal'] = 0
df.loc[df['SMA_5'] > df['SMA_10'], 'MA_Crossover_Signal'] = 1  # Bullish crossover
df.loc[df['SMA_5'] < df['SMA_10'], 'MA_Crossover_Signal'] = -1  # Bearish crossover
df['MA_Crossover'] = (df['SMA_5']-df['SMA_10'])/df['SMA_10']
df['MA_Crossover'] = df['MA_Crossover'].shift(1)
df['MA_Crossover_Signal'] = df['MA_Crossover_Signal'].shift(1)
df = df.drop(columns=['EMA_Close','SMA_5','SMA_10',"MA_Crossover"])

In [ ]:
fig = px.line(df, x="Date", y=["Stochastic_Scaled"])
fig.show()

In [ ]:

def fetch_and_concat_data(api_key):
    base_url = "https://api.twelvedata.com/time_series"
    #start_date = datetime(year=2023, month=12, day=1)
    start_date = datetime(year=2019, month=10, day=30)
    end_date = datetime.now()  # Or any other end date you want

    all_data_frames = []
    request_count = 0

    while start_date < end_date:
        params = {
            "symbol": "GDAXI",
            "interval": "5min",
            "format": "CSV",
            "apikey": api_key,
            "start_date": start_date.strftime("%m/%d/%Y 9:00"),
            "end_date": (start_date + timedelta(days=1)).strftime("%m/%d/%Y 17:31")
        }

        response = requests.get(base_url, params=params)
        request_count += 1

        if response.status_code == 200:
            csv_data = StringIO(response.text)
            df = pd.read_csv(csv_data, delimiter=';')
            df = df.iloc[::-1]
            all_data_frames.append(df)
        else:
            print(f"Failed to fetch data for {start_date.strftime('%Y-%m-%d')}")

        # Check if rate limit is reached
        if request_count >= 55:
            time.sleep(60)  # Sleep for 60 seconds
            request_count = 0  # Reset request count

        start_date += timedelta(days=1)

    # Concatenate all data frames
    final_df = pd.concat(all_data_frames, ignore_index=True)

    # Remove duplicates based on the datetime column
    final_df.drop_duplicates(subset='datetime', keep='first', inplace=True)

    return final_df

# Usage
api_key = "REDACTED_API_KEY"
data_frame = fetch_and_concat_data(api_key)

In [ ]:
intradaydax = data_frame[["datetime","open","high","low","close"]]
intradaydax.head()

In [ ]:
intradaydax['datetime'] = pd.to_datetime(intradaydax['datetime'])
intradaydax.set_index('datetime', inplace=True)

In [ ]:
# Function to calculate daily candles updated at every 5-minute interval
def calculate_updated_daily_candles(df):
    # Create an empty DataFrame for the updated daily candles
    updated_daily_candles = pd.DataFrame(columns=['open', 'high', 'low', 'close'], index=df.index)

    # Iterate over the 5-minute candles
    for current_time in df.index:
        # Filter data up to the current timestamp
        current_day_data = df[df.index.date == current_time.date()]
        up_to_current_time_data = current_day_data[current_day_data.index <= current_time]

        # Calculate updated daily candle
        updated_daily_candles.loc[current_time, 'open'] = current_day_data.iloc[0]['open']
        updated_daily_candles.loc[current_time, 'high'] = up_to_current_time_data['high'].max()
        updated_daily_candles.loc[current_time, 'low'] = up_to_current_time_data['low'].min()
        updated_daily_candles.loc[current_time, 'close'] = up_to_current_time_data.iloc[-1]['close']

    return updated_daily_candles

# Calculate the updated daily candles
intradaydax_agg = calculate_updated_daily_candles(intradaydax)

# Display the first few rows of the updated daily candles dataframe
intradaydax_agg.head()

In [ ]:
intradaydax_agg = intradaydax_agg.reset_index()

In [ ]:
intradaydax_agg.head()

In [ ]:
intradaydax_agg['Date'] = intradaydax_agg['datetime'].dt.date
intradaydax_agg['Date'] = pd.to_datetime(intradaydax_agg['Date'], errors='coerce')

In [ ]:
intradaydax_agg.head()

In [ ]:
intradaydax_agg[intradaydax_agg["Date"]== "2023-12-22"].iloc[0:17]

In [ ]:
intradaydax_agg =intradaydax_agg.rename(columns={"high": "High", "low": "Low","open": "Open","close": "Close"})

In [ ]:
def pricelong(indexvalue, knockoutprice) :
    """Calculate the price of long options """
    return np.maximum((indexvalue - knockoutprice) * 0.01, 0)

def priceshort(indexvalue, knockoutprice) :
    """Calculate the price of short options"""
    return np.maximum((knockoutprice - indexvalue) * 0.01, 0)

In [ ]:
 
def prepare_intra_day_data(date):
    """
    Prepares intra-day data for a given date.
    """
    # Load the data for the given date
    open_day = df[df["Date"] == date]["Open"]
    knockout_long = 0.97 * open_day
    knockout_short = 1.03 * open_day
    open_price_long = pricelong(open_day,knockout_long) 
    open_price_short = priceshort(open_day,knockout_short)
    df_intra_day = intradaydax_agg[intradaydax_agg["Date"]==date]
    temp_list = []
    for element in df_intra_day.iloc[0:17].iterrows():
        high_long = pricelong(element[1]["High"],knockout_long)
        low_long = pricelong(element[1]["Low"],knockout_long)
        close_long = pricelong(element[1]["Close"],knockout_long)

        high_short = priceshort(element[1]["High"],knockout_short)
        low_short = priceshort(element[1]["Low"],knockout_short)
        close_short = priceshort(element[1]["Close"],knockout_short)
        perc_high_long = (high_long - open_price_long) / open_price_long
        perc_low_long = (low_long - open_price_long) / open_price_long
        perc_close_long = (close_long - open_price_long) / open_price_long
        perc_high_short = (high_short - open_price_short) / open_price_short
        perc_low_short = (low_short - open_price_short) / open_price_short
        perc_close_short = (close_short - open_price_short) / open_price_short

        temp_list.append([perc_high_long.iloc[0],perc_low_long.iloc[0],perc_close_long.iloc[0],perc_high_short.iloc[0],perc_low_short.iloc[0],perc_close_short.iloc[0]])
   
    
    
    return temp_list

In [ ]:
test = []
test.append([12,23])
test.append([12,23233])

In [ ]:
np.array(test).shape

In [ ]:
np.array(prepare_intra_day_data("2023-12-22")).shape

In [ ]:
all_daily_data = []
for elem in intradaydax_agg["Date"].unique():
    all_daily_data.append(prepare_intra_day_data(elem))
all_daily_data = np.array(all_daily_data)

In [ ]:
all_daily_data.shape

In [ ]:
def findtarget(date):
    close_day = df[df["Date"] == date]["Close"].values[0]
    high_day = df[df["Date"] == date]["Close"].values[0]
    low_day = df[df["Date"] == date]["Close"].values[0]
    df_intra_day = intradaydax_agg[intradaydax_agg["Date"]==date]

    index_value = df_intra_day.iloc[19]["Close"]
    

    knockout_long = 0.97 * index_value
    knockout_short = 1.03 * index_value

    open_price_long = pricelong(index_value,knockout_long) 
     
    close_price_long = pricelong(close_day,knockout_long)
     

    if close_price_long > open_price_long:
        return 1
    else:
        return 0


    

In [ ]:
findtarget("2023-12-22")

In [ ]:
intradaydax_agg[intradaydax_agg["Date"]=="2023-12-22"]["Close"].max()

In [ ]:
close_day = df[df["Date"] == "2023-12-22"]["Close"].values[0]

In [ ]:
all_targets = []
for elem in intradaydax_agg["Date"].unique():
    all_targets.append(findtarget(elem))
all_targets = np.array(all_targets)

In [ ]:
all_targets.shape

In [ ]:
counts = np.bincount(all_targets)
num_zeros = counts[0]
num_ones = counts[1]

# Calculate proportions
total_elements = all_targets.size
proportion_zeros = num_zeros / total_elements
proportion_ones = num_ones / total_elements

# Print the results
print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)
print("Proportion of 0s:", proportion_zeros)
print("Proportion of 1s:", proportion_ones)

In [ ]:
def get_max_min_deviation(date):
    high_day = df[df["Date"] == date]["High"].values[0]
    low_day = df[df["Date"] == date]["Low"].values[0]
    high_day_intra_day = intradaydax_agg[intradaydax_agg["Date"] == date]["High"].max()
    low_day_intra_day = intradaydax_agg[intradaydax_agg["Date"] == date]["Low"].min()
    return [ np.abs((high_day-high_day_intra_day )/high_day),np.abs((low_day_intra_day-low_day)/low_day) ]

In [ ]:
get_max_min_deviation("2023-12-22")

In [ ]:
deviations = []
for elem in intradaydax_agg["Date"].unique():
    deviations.append(get_max_min_deviation(elem))


In [ ]:
deviations = np.array(deviations)

In [ ]:
deviations.shape

In [ ]:
max( deviations[:,0])

In [ ]:
np.sqrt(np.var( deviations[:,1]))

In [ ]:
close_day

In [ ]:
intradaydax_agg.tail()

In [ ]:
daily_df = intradaydax.resample('D').agg({'open': 'first', 
                                 'high': 'max', 
                                 'low': 'min', 
                                 'close': 'last'})

# Drop rows with NaN values (days where there might be no data)
daily_df.dropna(inplace=True)

In [ ]:
df['weekday'] = df["Date"].dt.weekday.astype(np.int8) 
df['month'] = df["Date"].dt.month.astype(np.int8) - 1
df['monthday'] = df["Date"].dt.day.astype(np.int8) -1
df['month_of_quarter'] = ((df["Date"].dt.month - 1) % 3) 
df = one_hot_encode_column(df,'weekday')
df = one_hot_encode_column(df,'month') 
df = one_hot_encode_column(df,'monthday')
df = one_hot_encode_column(df,'month_of_quarter')

In [ ]:
df.tail()

In [ ]:
columns = ['Open', 'High', 'Low',"Close",'Open_lag1', 'High_lag1', 'Low_lag1',"Close_lag1"]
columns_delete = []

for elem in columns:
    columns_delete.append(f"{elem}")
    columns_delete.append(f"{elem}_200_MA")
    columns_delete.append(f"{elem}_100_MA")
    columns_delete.append(f"{elem}_10_MA")


In [ ]:
df[columns_delete]

In [ ]:
columns_delete

In [ ]:
dataset= df.drop(columns_delete,axis=1)
dataset = dataset.iloc[:-1]
dataset = dataset.dropna()

In [ ]:
dataset.head()

In [ ]:
filtered_dataset = dataset[dataset['Date'].isin(intradaydax_agg['Date'])]

In [ ]:
dates_not_in_dataset = intradaydax_agg[~intradaydax_agg['Date'].isin(dataset['Date'])]

In [ ]:
dates_not_in_dataset

In [ ]:
data = filtered_dataset.values[:,1:]
data.shape

In [ ]:
filtered_dataset

In [ ]:
all_targets.shape

In [ ]:
all_daily_data.shape

In [ ]:
X1 = data
X2 = all_daily_data[:-1,:]
y = all_targets[:-1]

In [ ]:
y.shape

In [ ]:
X1.shape

In [ ]:
X2.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split each array individually while keeping the same random_state to ensure matching indices
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)
X2_train, X2_test, _ , _ = train_test_split(X2, y, test_size=0.2, random_state=42)  # y is just to keep the split consistent
data_train, data_test, _ , _ = train_test_split(filtered_dataset.values, y, test_size=0.2, random_state=42)  # y is just to keep the split consistent

In [ ]:
X1_train.shape

In [ ]:
X2_train.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout,LeakyReLU,BatchNormalization, concatenate
from tensorflow.keras.optimizers import Adam
from ncps import wirings
from ncps.tf import LTC
from tensorflow import keras

In [ ]:
fc_wiring = wirings.FullyConnected(64, 2)
fc_wiring2 = wirings.FullyConnected(64, 2)

In [ ]:
from keras.layers import LSTM
input_model_X1 = Input(shape= X1_train.shape[1:3])
denseone = Dense(64,)(input_model_X1)
denseone = LeakyReLU(alpha=0.3)(denseone)
denseone = Dropout(rate=0.3)(denseone)
denseone= BatchNormalization()(denseone)
    
densetwo = Dense(64,)(denseone)
densetwo = LeakyReLU(alpha=0.3)(densetwo)
densetwo = Dropout(rate=0.3)(densetwo)
densetwo= BatchNormalization()(densetwo)
final = Dense(1, name='Dense_final', activation='sigmoid')(densetwo)

input_model_X2 = Input(shape= X2_train.shape[1:3])
LTC(fc_wiring, return_sequences=True)(input_model_X2)
#lstm1 = LSTM(64, return_sequences=True)(input_model_X2)
#lstm1 = Dropout(rate=0.3)(lstm1)
#lstm1 = LeakyReLU(alpha=0.3)(lstm1)

lstm2 = LSTM(64)(lstm1)
lstm2 = Dropout(rate=0.3)(lstm2)
lstm2 = LeakyReLU(alpha=0.3)(lstm2)
final2 = Dense(1, name='Dense_final2', activation='sigmoid')(lstm2)

mergelayer = concatenate([final, final2])

densemerge = Dense(64)(mergelayer)
densemerge = LeakyReLU(alpha=0.3)(densemerge)
densemerge = Dropout(rate=0.3)(densemerge)
densemerge= BatchNormalization()(densemerge)
finalmerge = Dense(1, name='Dense_finalmerge', activation='sigmoid')(densemerge)

model = Model(inputs=[input_model_X1, input_model_X2], outputs=finalmerge)
model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=Adam())

In [ ]:
model.summary()

In [ ]:
X1_train = X1_train.astype(np.float32)
X2_train = X2_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X1_test = X1_test.astype(np.float32)
X2_test = X2_test.astype(np.float32)
y_test = y_test.astype(np.float32)


In [ ]:
history =model.fit([X1_train,X2_train],y_train,epochs=500,validation_data=([X1_test,X2_test], y_test),batch_size=32,verbose=0)

In [ ]:
y_test_predict =model.predict([X1_test,X2_test])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import plotly.graph_objects as go
import json

In [ ]:
bucketmodels = "REDACTED_BUCKET"

In [ ]:
def upload_file_to_s3(bucket, key, file):
    s3 = boto3.client("s3")
    s3.upload_file(file, bucket, key)
    

In [ ]:
def write_data_to_s3(bucket, key, content):
    s3 = boto3.client("s3")
    s3.put_object(Bucket=bucket, Key=key, Body=content)
     

In [ ]:
def evaluate_strategy(date, direction):
    close_day = df[df["Date"] == date]["Close"].values[0]
    high_day = df[df["Date"] == date]["Close"].values[0]
    low_day = df[df["Date"] == date]["Close"].values[0]
    df_intra_day = intradaydax_agg[intradaydax_agg["Date"]==date]

    index_value = df_intra_day.iloc[19]["Close"]
    

    knockout_long = 0.97 * index_value
    knockout_short = 1.03 * index_value

    open_price_long = pricelong(index_value,knockout_long)
    open_price_short = priceshort(index_value,knockout_short) 
     
    close_price_long = pricelong(close_day,knockout_long)
    close_price_short = priceshort(close_day,knockout_short)

    if close_price_long < 0.71 *open_price_long:
        close_price_long = 0.71 *open_price_long
    if close_price_short < 0.71 *open_price_short:
        close_price_short = 0.71 *open_price_short 
    
    if direction == 1:
        return 1+ (close_price_long - open_price_long) / open_price_long
    else:
        return 1+(close_price_short - open_price_short) / open_price_short
     

  

In [ ]:
TOTAL_ITERATIONS = 10000
LOG_INTERVAL = 100
start_time = time.time()
def optimizeml(**kwargs):
    
    tf.keras.backend.clear_session()
    global performance, prediction, counter
    p1 = int(kwargs["x1"])
    drop1 = kwargs["drop1"]
    l1 = kwargs["leaky1"]
   
    p2 = int(kwargs["x2"])
    l2 = kwargs["leaky2"]
    drop2 = kwargs["drop2"]
    
    p3 = int(kwargs["x3"])
    l3 = kwargs["leaky3"]
    drop3 = kwargs["drop3"]
    
    lstm1p1 = int(kwargs["lstm1"])
    lstml1 = kwargs["lstmleaky1"]
    lstm1drop1 = kwargs["lstmdrop1"]
    
    lstm2p2 = int(kwargs["lstm2"])
    lstml2 = kwargs["lstmleaky2"]
    lstm2drop2 = kwargs["lstmdrop2"]
    
    epoch =int(kwargs["epoch"])
    batch_size =int(kwargs["batch_size"])
    
    input_model_X1 = Input(shape= X1_train.shape[1:3])
    denseone = Dense(p1,)(input_model_X1)
    denseone = LeakyReLU(alpha=l1)(denseone)
    denseone = Dropout(rate=drop1)(denseone)
    denseone= BatchNormalization()(denseone)
    
    densetwo = Dense(p2,)(denseone)
    densetwo = LeakyReLU(alpha=l2)(densetwo)
    densetwo = Dropout(rate=drop2)(densetwo)
    densetwo= BatchNormalization()(densetwo)
    final = Dense(1, name='Dense_final', activation='sigmoid')(densetwo)

    input_model_X2 = Input(shape= X2_train.shape[1:3])
    lstm1 = LSTM(lstm1p1, return_sequences=True)(input_model_X2)
    lstm1 = Dropout(rate=lstml1)(lstm1)
    lstm1 = LeakyReLU(alpha=lstm1drop1)(lstm1)

    lstm2 = LSTM(lstm2p2)(lstm1)
    lstm2 = Dropout(rate=lstm2drop2)(lstm2)
    lstm2 = LeakyReLU(alpha=lstml2)(lstm2)
    final2 = Dense(1, name='Dense_final2', activation='sigmoid')(lstm2)

    mergelayer = concatenate([final, final2])

    densemerge = Dense(p3)(mergelayer)
    densemerge = LeakyReLU(alpha=l3)(densemerge)
    densemerge = Dropout(rate=drop3)(densemerge)
    densemerge= BatchNormalization()(densemerge)
    finalmerge = Dense(1, name='Dense_finalmerge', activation='sigmoid')(densemerge)

    model = Model(inputs=[input_model_X1, input_model_X2], outputs=finalmerge)
    model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=Adam())
    
    
    history =model.fit([X1_train,X2_train],y_train,epochs=epoch,validation_data=([X1_test,X2_test], y_test),batch_size=batch_size,verbose=0)
    loss = history.history["val_loss"]
    last_val_loss = loss[len(loss)-1]
    y_test_predict =model.predict([X1_test,X2_test],verbose=0)
    y_pred_class = (y_test_predict > 0.5).astype("int32")  # Convert probabilities to class labels
    fpr, tpr, thresholds = roc_curve(y_test, y_test_predict)
    roc_auc = float(auc(fpr, tpr))
    
    strat_performance = 1
    for i, elem in enumerate(data_test[:,0]):
        direction = y_pred_class[i,0]
        temp_performance = evaluate_strategy(elem, direction)
        strat_performance *= temp_performance
    


    if strat_performance > performance:
        logger.info(f"New performance is {strat_performance} and it is better than the previous one {performance} ")  
        performance = strat_performance
        model.save_weights(f"interdaymodel.h5f", overwrite=True)
        upload_file_to_s3(bucketmodels,f"confidencemodelday/interdaymodel.h5f.data-00000-of-00001", f"interdaymodel.h5f.data-00000-of-00001")
        upload_file_to_s3(bucketmodels,f"confidencemodelday/interdaymodel.h5f.index",f"interdaymodel.h5f.index")
        write_data_to_s3(bucketmodels, f"confidencemodelday/interdaymodel.csv", json.dumps(kwargs ))
        
    counter += 1
    if counter % LOG_INTERVAL == 0:
        elapsed_time = time.time() - start_time
        avg_speed = counter / elapsed_time * 60  # iterations per minute
        
        remaining_iterations = TOTAL_ITERATIONS - counter
        estimated_completion_time = remaining_iterations / avg_speed  /60
        
        logger.info(f"Completed {counter/TOTAL_ITERATIONS*100}% of the iterations.")
        logger.info(f"Average speed: {avg_speed:.2f} iterations/minute.")
        logger.info(f"Estimated time to completion: {estimated_completion_time:.2f} hours.")
    return strat_performance

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
from bayes_opt import BayesianOptimization
optimizer = BayesianOptimization(
    f=optimizeml,
     pbounds={'x1': (1, 128),'x2': (1, 128),'x3': (1, 128),'x4': (1, 128),
 'drop1': (0, 0.5),'drop2': (0, 0.5),'drop3': (0, 0.5),'drop4': (0, 0.5),
 'leaky1': (0, 1),'leaky2': (0, 1),'leaky3': (0, 1),'leaky4': (0, 1),
 "lstm1": (1, 128),'lstm2': (1, 128),
 "lstmleaky1" : (0, 1),'lstmleaky2': (0, 1),
 "lstmdrop1": (0, 0.5),'lstmdrop2': (0, 0.5),
 'epoch': (10, 200),
 'batch_size': (2, 256)},
    verbose=0,
    random_state=42,
    #bounds_transformer=bounds_transformer
)
performance = 0
counter = 0
optimizer.maximize(
    init_points=5,
    n_iter=TOTAL_ITERATIONS-5,
)

In [ ]:
def load_model(kwargs):
    
    tf.keras.backend.clear_session()
     
    p1 = int(kwargs["x1"])
    drop1 = kwargs["drop1"]
    l1 = kwargs["leaky1"]
   
    p2 = int(kwargs["x2"])
    l2 = kwargs["leaky2"]
    drop2 = kwargs["drop2"]
    
    p3 = int(kwargs["x3"])
    l3 = kwargs["leaky3"]
    drop3 = kwargs["drop3"]
    
    lstm1p1 = int(kwargs["lstm1"])
    lstml1 = kwargs["lstmleaky1"]
    lstm1drop1 = kwargs["lstmdrop1"]
    
    lstm2p2 = int(kwargs["lstm2"])
    lstml2 = kwargs["lstmleaky2"]
    lstm2drop2 = kwargs["lstmdrop2"]
    
    epoch =int(kwargs["epoch"])
    batch_size =int(kwargs["batch_size"])
    
    input_model_X1 = Input(shape= X1_train.shape[1:3])
    denseone = Dense(p1,)(input_model_X1)
    denseone = LeakyReLU(alpha=l1)(denseone)
    denseone = Dropout(rate=drop1)(denseone)
    denseone= BatchNormalization()(denseone)
    
    densetwo = Dense(p2,)(denseone)
    densetwo = LeakyReLU(alpha=l2)(densetwo)
    densetwo = Dropout(rate=drop2)(densetwo)
    densetwo= BatchNormalization()(densetwo)
    final = Dense(1, name='Dense_final', activation='sigmoid')(densetwo)

    input_model_X2 = Input(shape= X2_train.shape[1:3])
    lstm1 = LSTM(lstm1p1, return_sequences=True)(input_model_X2)
    lstm1 = Dropout(rate=lstml1)(lstm1)
    lstm1 = LeakyReLU(alpha=lstm1drop1)(lstm1)

    lstm2 = LSTM(lstm2p2)(lstm1)
    lstm2 = Dropout(rate=lstm2drop2)(lstm2)
    lstm2 = LeakyReLU(alpha=lstml2)(lstm2)
    final2 = Dense(1, name='Dense_final2', activation='sigmoid')(lstm2)

    mergelayer = concatenate([final, final2])

    densemerge = Dense(p3)(mergelayer)
    densemerge = LeakyReLU(alpha=l3)(densemerge)
    densemerge = Dropout(rate=drop3)(densemerge)
    densemerge= BatchNormalization()(densemerge)
    finalmerge = Dense(1, name='Dense_finalmerge', activation='sigmoid')(densemerge)

    model = Model(inputs=[input_model_X1, input_model_X2], outputs=finalmerge)
    model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=Adam())
    model.load_weights("interdaymodel.h5f")
    return model

In [ ]:
s3 = boto3.resource('s3')

In [ ]:
dax = (s3.Object("REDACTED_BUCKET", "confidencemodelday/interdaymodel.csv").get()['Body'].read().decode('utf-8') )
analysisfile = open("interdaymodel.csv", "w+")
analysisfile.write(dax)
analysisfile.close()
dax = (s3.Object("REDACTED_BUCKET", "confidencemodelday/interdaymodel.h5f.index").get()['Body'].read() )
analysisfile = open("interdaymodel.h5f.index", 'wb')
analysisfile.write(dax)
analysisfile.close()
dax = (s3.Object("REDACTED_BUCKET", "confidencemodelday/interdaymodel.h5f.data-00000-of-00001").get()['Body'].read() )
analysisfile = open("interdaymodel.h5f.data-00000-of-00001", 'wb')
analysisfile.write(dax)
analysisfile.close()

In [ ]:
schema = json.loads(open("interdaymodel.csv", 'r').read())
schema
   

In [ ]:
model = load_model(schema) 

In [ ]:
model.summary()

In [ ]:
y_test_predict =model.predict([X1_test,X2_test],verbose=0)
y_pred_class = (y_test_predict > 0.5).astype("int32")

In [ ]:
y_test_predict[1,0]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_class)
roc_auc = auc(fpr, tpr)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC curve (area = %0.2f)' % roc_auc))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='No Skill'))
fig.update_layout(xaxis_title='False Positive Rate', yaxis_title='True Positive Rate', title='Receiver Operating Characteristic')
fig.show()

In [ ]:
evaluate_strategy("2023-12-12", 0)

In [ ]:
performance = 1
for i, elem in enumerate(data_test[:,0]):
    direction = y_pred_class[i,0]
    temp_performance = evaluate_strategy(elem, direction)
    performance *= temp_performance
    


In [ ]:
performance

In [ ]:
X2_train.shape[1:3]

In [ ]:
roc_auc = roc_auc_score(y_test, y_test_predict)

In [ ]:
print(f"ROC AUC: {roc_auc}")

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_test_predict)
roc_auc = auc(fpr, tpr)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC curve (area = %0.2f)' % roc_auc))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='No Skill'))
fig.update_layout(xaxis_title='False Positive Rate', yaxis_title='True Positive Rate', title='Receiver Operating Characteristic')
fig.show()

# Let's try a more subtle approach

In [ ]:
X_train.shape[1:3]

In [ ]:
X1[len(X1)-1]

In [ ]:
X2[len(X1)-1]